In [1]:
import pandas as pd

In [5]:
ranking = '../evaluation/resources/2020/jsonruns/p_controller_default_nle_meta_9_1_theta=0.9.json'
eval_result = '../evaluation/resources/2020/eval_results/p_controller_default_nle_meta_9_1_theta=0.9.tsv'

In [32]:
gt = '../pre_processing/resources/evaluation/2020/TREC-Fair-Ranking-eval-sample.json'

In [6]:
rdf= pd.read_json(ranking,lines=True)

In [38]:
gtdf = pd.read_json(gt, lines=True).explode('documents')
gtdf['doc_id'] = gtdf.documents.apply(lambda row: row.get('doc_id'))
gtdf['doc_rel'] = gtdf.documents.apply(lambda row: row.get('relevance'))
reldf = gtdf[['qid','query','doc_id','doc_rel']]
reldf

,qid,query,doc_id,doc_rel
0,9087,lepidium sativum,40d8bea494d4ff4c46f7ad5cb2dcdaa6f96ed5a4,1
0,9087,lepidium sativum,008a2a2c765485af2e035a2572ba9fc1102eabff,1
0,9087,lepidium sativum,4a1472c575488ed9acee6c5481f68b95a0c34092,0
0,9087,lepidium sativum,ff8df0745136fb183ea72b783ff38f8a7ad5ceec,0
0,9087,lepidium sativum,90605deb1da5e3d5548da12dc130fc0341a6a255,0
0,9087,lepidium sativum,cbc33164f9db97698901f130f399f2898d856267,0
0,9087,lepidium sativum,6b70f398278146116baa4865582c36f5550f953a,0
0,9087,lepidium sativum,c68374b97eb2c214afd3c282175f664fc37c5682,0
0,9087,lepidium sativum,34ba4f3d2a38b88153308f2a7f12651b440f5812,0
0,9087,lepidium sativum,efe0b6115d460fcd2173fc04d113694f1ded8692,0


In [11]:
edf = pd.read_csv(eval_result, sep='\t', names=['key', 'qid', 'value'])
edf = edf.pivot(index='qid', columns='key', values='value')

In [15]:
edf.sort_values(by='difference')

key,difference,disparity,relevance
qid,,,
91643,0.019822,1.006921,0.868206
5191,0.044683,1.448939,1.344147
81479,0.046552,2.044160,1.889465
21237,0.053041,1.358547,1.149610
7363,0.053171,1.313280,1.085770
18112,0.057675,1.327767,1.079491
61688,0.061505,1.061153,0.834400
4319,0.063171,1.892463,1.585866
43723,0.065366,1.839158,1.496630


In [19]:
edf[edf.difference > edf.difference.mean()].sort_values(by='difference',ascending=False).head(10)

key,difference,disparity,relevance
qid,,,
657,3.700546,2.631709,0.255715
100510,3.252370,2.703133,0.514383
16607,3.034450,2.557715,0.550634
89881,1.987253,1.570899,0.577918
12885,1.941179,1.874742,0.848831
18060,1.725684,1.519402,0.684165
59483,1.619563,1.586472,0.469430
52501,1.429890,1.625181,0.943276
89569,1.210698,1.378091,0.572047


In [24]:
query_results = rdf[rdf.qid == 657]
query_results

,q_num,qid,ranking
24150,161.000,657,"[1436449e0234166b13135df45be1f072df059007, 863..."
24151,161.100,657,"[18f883bd5b9ab9771e561a0214af6bd5ddda0d61, 863..."
24152,161.200,657,"[e9cfbf58d87e9f5afae813e8b8294b7719c176da, b79..."
24153,161.300,657,"[128611b63b268a1bffb69d3276a11ae7f6b068d7, 3c9..."
24154,161.400,657,"[e686c4e74d9085c78813355d582102d973b157cf, b79..."
24155,161.500,657,"[a8b58f310e13c171ce7b7890d23860836cbc4534, 143..."
24156,161.600,657,"[863a0360e4345b2e106e4f44cd95821f68b28036, 3c9..."
24157,161.700,657,"[87596955211a23714a6c2fc2af2d00464d15f9b3, 480..."
24158,161.800,657,"[3589a8b05684193dfc55610c9a9fbcf6db490130, 147..."
24159,161.900,657,"[6b7ca72ca79251cded87c47608e5cc93e37b09f2, b79..."


# optie 4

How often is a rel file in a position versus a nonrel file?

In [29]:
# aggregate the position rel/nonrel counts
df = pd.DataFrame({'rank':list(range(1,len(query_results['ranking'].iloc[0])))})

In [47]:
reldf[(reldf.qid == 657) & (reldf.doc_id == '1436449e0234166b13135df45be1f072df059007')].doc_rel.iloc[0]

0

In [52]:
countdict = {}

for idx, row in query_results.iterrows():
    for i,doc in enumerate(row['ranking']):
        if not i + 1 in countdict:
            countdict[i+1] = {'r':0,'nr':0}
        rel = reldf[(reldf.qid == 657) & (reldf.doc_id == doc)].doc_rel.iloc[0]
        if rel == 0:
            countdict[i+1]['nr'] = countdict[i+1]['nr'] + 1
        elif rel == 1:
            countdict[i+1]['r'] = countdict[i+1]['r'] + 1
        else:
            raise ValueError
        
        
        
        

In [53]:
countdict

{1: {'r': 1, 'nr': 149},
 2: {'r': 7, 'nr': 143},
 3: {'r': 4, 'nr': 146},
 4: {'r': 3, 'nr': 147},
 5: {'r': 4, 'nr': 146},
 6: {'r': 3, 'nr': 147},
 7: {'r': 5, 'nr': 145},
 8: {'r': 4, 'nr': 146},
 9: {'r': 7, 'nr': 143},
 10: {'r': 3, 'nr': 147},
 11: {'r': 7, 'nr': 143},
 12: {'r': 3, 'nr': 147},
 13: {'r': 6, 'nr': 144},
 14: {'r': 6, 'nr': 144},
 15: {'r': 4, 'nr': 146},
 16: {'r': 4, 'nr': 146},
 17: {'r': 6, 'nr': 144},
 18: {'r': 6, 'nr': 144},
 19: {'r': 11, 'nr': 139},
 20: {'r': 11, 'nr': 139},
 21: {'r': 4, 'nr': 146},
 22: {'r': 7, 'nr': 143},
 23: {'r': 8, 'nr': 142},
 24: {'r': 8, 'nr': 142},
 25: {'r': 7, 'nr': 143},
 26: {'r': 11, 'nr': 139},
 27: {'r': 9, 'nr': 141},
 28: {'r': 10, 'nr': 140},
 29: {'r': 6, 'nr': 144},
 30: {'r': 10, 'nr': 140},
 31: {'r': 5, 'nr': 145},
 32: {'r': 8, 'nr': 142},
 33: {'r': 9, 'nr': 141},
 34: {'r': 10, 'nr': 140},
 35: {'r': 9, 'nr': 141},
 36: {'r': 15, 'nr': 135},
 37: {'r': 19, 'nr': 131},
 38: {'r': 14, 'nr': 136},
 39: {'r': 1

In [62]:
rankdf = pd.DataFrame([{**{'rank':k},**v}  for k,v in countdict.items()])
rankdf.head()

,rank,r,nr
0,1,1,149
1,2,7,143
2,3,4,146
3,4,3,147
4,5,4,146


In [63]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks", color_codes=True)
titanic = sns.load_dataset("titanic")


In [64]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False
